In [2]:
import datetime

print(datetime.datetime.now())

2024-09-17 16:25:07.677290


**Background and question**: I have never benchmarked an AI model. I installed a Mistral model back in 02/24 for a separate high-level idea. Can I benchmark this model under an hour?

**Response from 9/23:** The answer is no, especially given the benchmark I picked.

In [2]:
import json

with open("./ARC-AGI-master/data/training/0a938d79.json") as f:
    some_file = json.load(f)
    print(some_file)

{'train': [{'input': [[0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'output': [[0, 0, 0, 0, 0, 2, 0, 8, 0, 2, 0, 8, 0, 2, 0, 8, 0, 2, 0, 8, 0, 2, 0, 8, 0], [0, 0, 0, 0, 0, 2, 0, 8, 0, 2, 0, 8, 0, 2, 0, 8, 0, 2, 0, 8, 0, 2, 0, 8, 0], [0, 0, 0, 0, 0, 2, 0, 8, 0, 2, 0, 8, 0, 2,

In [2]:
# reusing code from earlier
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch

In [6]:
model_name = "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        #load_in_4bit=True,
        #quantization_config=bnb_config,
        torch_dtype=torch.float16, #bfloat if quantization was working
        device_map="mps",
        #trust_remote_code=True,
    )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
params_7b = model.num_parameters() / 1_000_000
print(round(params_7b), "million")

In [3]:
import os 

matches = []
train_file_names = []
for root, dirnames, filenames in os.walk("./ARC-AGI-master/data/training"):
    for filename in filenames:
        if filename.endswith('.json'):
            matches.append(os.path.join(root, filename))
            train_file_names.append(filename)
            
print(matches)

['./ARC-AGI-master/data/training/a85d4709.json', './ARC-AGI-master/data/training/c8cbb738.json', './ARC-AGI-master/data/training/8e1813be.json', './ARC-AGI-master/data/training/a699fb00.json', './ARC-AGI-master/data/training/5c2c9af4.json', './ARC-AGI-master/data/training/44f52bb0.json', './ARC-AGI-master/data/training/23581191.json', './ARC-AGI-master/data/training/94f9d214.json', './ARC-AGI-master/data/training/f9012d9b.json', './ARC-AGI-master/data/training/4258a5f9.json', './ARC-AGI-master/data/training/bdad9b1f.json', './ARC-AGI-master/data/training/d06dbe63.json', './ARC-AGI-master/data/training/8403a5d5.json', './ARC-AGI-master/data/training/67e8384a.json', './ARC-AGI-master/data/training/8731374e.json', './ARC-AGI-master/data/training/25ff71a9.json', './ARC-AGI-master/data/training/ecdecbb3.json', './ARC-AGI-master/data/training/6e19193c.json', './ARC-AGI-master/data/training/39e1d7f9.json', './ARC-AGI-master/data/training/ba97ae07.json', './ARC-AGI-master/data/training/99b1bc4

In [4]:
import pandas as pd

bunch_of_series = []
for name in train_file_names:
    with (open(f"./ARC-AGI-master/data/training/{name}") as temp_file):
        some_file = json.load(temp_file)
        
        try:
            prob_ex_3_placement_input = [[str(i[j]) for i in 
                                         some_file["train"][2]["input"]] for j in range
                                         (len(some_file["train"][2]["input"][0]))]
            prob_ex_3_placement_output = [[str(i[j]) for i in 
                                         some_file["train"][2]["output"]] for j in range
                                         (len(some_file["train"][2]["output"][0]))]
            #some_file["train"][2]
        except IndexError:
            prob_ex_3_placement = {"input": "", "output": ""}
            
        try:
            prob_ex_4_placement_input = [[str(i[j]) for i in 
                                         some_file["train"][3]["input"]] for j in range
                                         (len(some_file["train"][3]["input"][0]))]
            prob_ex_4_placement_output = [[str(i[j]) for i in 
                                         some_file["train"][3]["output"]] for j in range
                                         (len(some_file["train"][3]["output"][0]))]
            # some_file["train"][3]
        except IndexError:
            prob_ex_4_placement = {"input": [""], "output": [""]}
        
        temp_series = pd.Series({
            "problem_id": name,
            "problem_example_1_input":  [[str(i[j]) for i in 
                                         some_file["train"][0]["input"]] for j in range
                                         (len(some_file["train"][0]["input"][0]))],
            "problem_example_1_output": [[str(i[j]) for i in 
                                         some_file["train"][0]["output"]] for j in range
                                         (len(some_file["train"][0]["output"][0]))],
            "problem_example_2_input":  [[str(i[j]) for i in 
                                         some_file["train"][1]["input"]] for j in range
                                         (len(some_file["train"][1]["input"][0]))],
            "problem_example_2_output": [[str(i[j]) for i in 
                                         some_file["train"][1]["output"]] for j in range
                                         (len(some_file["train"][1]["output"][0]))],
            "problem_example_3_input":  prob_ex_3_placement_input,
            "problem_example_3_output": prob_ex_3_placement_output,
            "problem_example_4_input":  prob_ex_4_placement_input, 
            "problem_example_4_output": prob_ex_4_placement_output,
            "problem_test_input":       [[str(i[j]) for i in 
                                         some_file["test"][0]["input"]] for j in range
                                         (len(some_file["test"][0]["input"][0]))],
            "problem_test_output":      [[str(i[j]) for i in 
                                         some_file["test"][0]["output"]] for j in range
                                         (len(some_file["test"][0]["output"][0]))],
        })
        
        bunch_of_series.append(temp_series)

In [5]:
some_file["train"]

[{'input': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0],
   [0, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0],
   [0, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
  'output': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0],
   [0, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0],
   [0, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 

In [6]:
#test_df

In [7]:
arc_train = pd.concat(bunch_of_series, axis=1).T
arc_train

,problem_id,problem_example_1_input,problem_example_1_output,problem_example_2_input,problem_example_2_output,problem_example_3_input,problem_example_3_output,problem_example_4_input,problem_example_4_output,problem_test_input,problem_test_output
0,a85d4709.json,"[[0, 0, 5], [0, 5, 0], [5, 0, 0]]","[[3, 4, 2], [3, 4, 2], [3, 4, 2]]","[[0, 0, 0], [0, 0, 0], [5, 5, 5]]","[[3, 3, 3], [3, 3, 3], [3, 3, 3]]","[[5, 0, 5], [0, 5, 0], [0, 0, 0]]","[[2, 4, 2], [2, 4, 2], [2, 4, 2]]","[[0, 0, 0], [5, 0, 5], [0, 5, 0]]","[[4, 3, 4], [4, 3, 4], [4, 3, 4]]","[[0, 5, 0], [0, 0, 5], [5, 0, 0]]","[[3, 2, 4], [3, 2, 4], [3, 2, 4]]"
1,c8cbb738.json,"[[3, 3, 3, 3, 3, 3, 3, 3, 8, 3, 8, 3], [3, 3, ...","[[2, 8, 1, 8, 2], [4, 3, 3, 3, 4], [1, 3, 3, 3...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 8, 1, 8, ...","[[8, 3, 8], [3, 1, 3], [8, 3, 8]]","[[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4], [4, 4, ...","[[1, 4, 7, 4, 1], [4, 4, 4, 4, 4], [7, 4, 4, 4...","[[0, 0, 0], [5, 0, 5], [0, 5, 0]]","[[4, 3, 4], [4, 3, 4], [4, 3, 4]]","[[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,...","[[3, 8, 2, 1, 2, 8, 3], [8, 8, 8, 8, 8, 8, 8],..."
2,8e1813be.json,"[[0, 0, 2, 0, 0, 1, 0, 0, 3, 0, 0, 0, 0, 0, 0,...","[[2, 1, 3, 4, 8, 6], [2, 1, 3, 4, 8, 6], [2, 1...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 5, ...","[[1, 1, 1], [2, 2, 2], [4, 4, 4]]","[[0, 2, 0, 0, 8, 0, 0, 4, 0, 0, 1, 0], [0, 2, ...","[[2, 8, 4, 1], [2, 8, 4, 1], [2, 8, 4, 1], [2,...","[[0, 0, 0], [5, 0, 5], [0, 5, 0]]","[[4, 3, 4], [4, 3, 4], [4, 3, 4]]","[[2, 0, 0, 3, 0, 0, 8, 0, 0, 4, 0, 0, 6, 0, 0,...","[[2, 3, 8, 4, 6, 1, 7], [2, 3, 8, 4, 6, 1, 7],..."
3,a699fb00.json,"[[1, 0, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0...","[[1, 0, 0, 0, 0], [2, 0, 0, 1, 0], [1, 0, 0, 2...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, ...","[[0, 0, 0], [5, 0, 5], [0, 5, 0]]","[[4, 3, 4], [4, 3, 4], [4, 3, 4]]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, ..."
4,5c2c9af4.json,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,...","[[0, 0, 0], [5, 0, 5], [0, 5, 0]]","[[4, 3, 4], [4, 3, 4], [4, 3, 4]]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...,...,...,...,...,...,...,...,...,...
395,7b7f7511.json,"[[1, 1, 3, 2], [1, 1, 3, 3], [3, 3, 1, 1], [2,...","[[1, 1, 3, 2], [1, 1, 3, 3], [3, 3, 1, 1], [2,...","[[4, 6, 6], [4, 4, 6], [4, 8, 8], [4, 6, 6], [...","[[4, 6, 6], [4, 4, 6], [4, 8, 8]]","[[2, 3, 4, 2, 3, 4], [3, 2, 4, 3, 2, 4]]","[[2, 3, 4], [3, 2, 4]]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, ...","[[3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [3, 3, 3, 3, ...","[[5, 4, 6, 2, 5, 4, 6, 2], [4, 5, 6, 6, 4, 5, ...","[[5, 4, 6, 2], [4, 5, 6, 6], [5, 4, 4, 2]]"
396,8be77c9e.json,"[[1, 1, 0], [1, 1, 0], [0, 1, 0]]","[[1, 1, 0, 0, 1, 1], [1, 1, 0, 0, 1, 1], [0, 1...","[[0, 1, 1], [0, 0, 1], [0, 1, 0]]","[[0, 1, 1, 1, 1, 0], [0, 0, 1, 1, 0, 0], [0, 1...","[[0, 0, 0], [0, 0, 0], [0, 1, 1]]","[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 1...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, ...","[[3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [3, 3, 3, 3, ...","[[0, 0, 1], [0, 0, 0], [0, 1, 0]]","[[0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0], [0, 1..."
397,dae9d2b5.json,"[[4, 4, 0], [4, 0, 0], [0, 0, 0], [3, 3, 0], [...","[[6, 6, 0], [6, 0, 0], [0, 0, 6]]","[[4, 4, 0], [0, 0, 0], [4, 0, 4], [3, 3, 3], [...","[[6, 6, 6], [6, 0, 0], [6, 0, 6]]","[[0, 0, 4], [0, 4, 4], [4, 4, 0], [0, 3, 0], [...","[[0, 6, 6], [6, 6, 6], [6, 6, 6]]","[[4, 0, 4], [4, 0, 0], [0, 0, 0], [3, 0, 0], [...","[[6, 0, 6], [6, 0, 0], [0, 6, 0]]","[[0, 4, 0], [4, 0, 4]

In [8]:
from datasets import Dataset

#arc_train_hf = Dataset.from_pandas(arc_train)

Things I have done so far:
1. Downloaded the ARC prize benchmark
2. Played around with it; I understand the data structure better now
3. Load the model and basic code
4. Convert the json training files to a dataframe
5. **The plan**
  - Tokenize the dataset
  - Finetune the model weights based on the tokenized dataset
  - Test the finetuned model on the evaluation set


In [73]:
# CHECKPOINT 
import datetime
print(datetime.datetime.now())

2024-09-17 17:44:29.061345


In [75]:
# restarting work on this
import datetime
print(datetime.datetime.now())

2024-09-18 11:58:06.441338


In [112]:
arc_train2 = arc_train.astype(str)
arc_train2["problem_example_1_input"].iloc[0]

'[[[0, 0, 5], [0, 5, 0], [5, 0, 0]]]'

In [113]:
tokenizer(arc_train2["problem_example_1_input"].iloc[0])

{'input_ids': [1, 8070, 28792, 28734, 28725, 28705, 28734, 28725, 28705, 28782, 1181, 733, 28734, 28725, 28705, 28782, 28725, 28705, 28734, 1181, 733, 28782, 28725, 28705, 28734, 28725, 28705, 28734, 7700, 28793], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [194]:
testl = [[[0, 0, 5], [0, 5, 0], [5, 0, 0]]]
testl = [[str(testl[0][j][i]) for i in range(3)] for j in range(3)]

In [201]:
tokenizer(testl[0])

{'input_ids': [[1, 28705, 28734], [1, 28705, 28734], [1, 28705, 28782]], 'attention_mask': [[1, 1, 1], [1, 1, 1], [1, 1, 1]]}

In [31]:
arc_no_id = arc_train.loc[:, arc_train.columns != "problem_id"]

In [15]:
#def convert_num2str(row):
#    total_range1 = len(row[0][0])
#    total_range2 = len(row[0])
#    return [[str(row[0][j][i]) for i in range(total_range1)] for j in range
#    (total_range2)][0]

#arc_no_id.map(convert_num2str)

In [11]:
arc_no_id

,problem_example_1_input,problem_example_1_output,problem_example_2_input,problem_example_2_output,problem_example_3_input,problem_example_3_output,problem_example_4_input,problem_example_4_output,problem_test_input,problem_test_output
0,"[[0, 0, 5], [0, 5, 0], [5, 0, 0]]","[[3, 4, 2], [3, 4, 2], [3, 4, 2]]","[[0, 0, 0], [0, 0, 0], [5, 5, 5]]","[[3, 3, 3], [3, 3, 3], [3, 3, 3]]","[5, 0, 0, 0, 5, 0, 5, 0, 0]","[2, 2, 2, 4, 4, 4, 2, 2, 2]","[[0, 0, 0], [5, 0, 5], [0, 5, 0]]","[[4, 3, 4], [4, 3, 4], [4, 3, 4]]","[[0, 5, 0], [0, 0, 5], [5, 0, 0]]","[[3, 2, 4], [3, 2, 4], [3, 2, 4]]"
1,"[[3, 3, 3, 3, 3, 3, 3, 3, 8, 3, 8, 3], [3, 3, ...","[[2, 8, 1, 8, 2], [4, 3, 3, 3, 4], [1, 3, 3, 3...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 8, 1, 8, ...","[[8, 3, 8], [3, 1, 3], [8, 3, 8]]","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[1, 4, 7, 4, 1, 4, 4, 4, 4, 4, 7, 4, 4, 4, 7, ...","[[0, 0, 0], [5, 0, 5], [0, 5, 0]]","[[4, 3, 4], [4, 3, 4], [4, 3, 4]]","[[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,...","[[3, 8, 2, 1, 2, 8, 3], [8, 8, 8, 8, 8, 8, 8],..."
2,"[[0, 0, 2, 0, 0, 1, 0, 0, 3, 0, 0, 0, 0, 0, 0,...","[[2, 1, 3, 4, 8, 6], [2, 1, 3, 4, 8, 6], [2, 1...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 5, ...","[[1, 1, 1], [2, 2, 2], [4, 4, 4]]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, ...","[2, 2, 2, 2, 8, 8, 8, 8, 4, 4, 4, 4, 1, 1, 1, 1]","[[0, 0, 0], [5, 0, 5], [0, 5, 0]]","[[4, 3, 4], [4, 3, 4], [4, 3, 4]]","[[2, 0, 0, 3, 0, 0, 8, 0, 0, 4, 0, 0, 6, 0, 0,...","[[2, 3, 8, 4, 6, 1, 7], [2, 3, 8, 4, 6, 1, 7],..."
3,"[[1, 0, 0, 0, 0], [0, 0, 0, 1, 0], [1, 0, 0, 0...","[[1, 0, 0, 0, 0], [2, 0, 0, 1, 0], [1, 0, 0, 2...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0, 0, 0], [5, 0, 5], [0, 5, 0]]","[[4, 3, 4], [4, 3, 4], [4, 3, 4]]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, ..."
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[[0, 0, 0], [5, 0, 5], [0, 5, 0]]","[[4, 3, 4], [4, 3, 4], [4, 3, 4]]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...,...,...,...,...,...,...,...,...
395,"[[1, 1, 3, 2], [1, 1, 3, 3], [3, 3, 1, 1], [2,...","[[1, 1, 3, 2], [1, 1, 3, 3], [3, 3, 1, 1], [2,...","[[4, 6, 6], [4, 4, 6], [4, 8, 8], [4, 6, 6], [...","[[4, 6, 6], [4, 4, 6], [4, 8, 8]]","[2, 3, 3, 2, 4, 4, 2, 3, 3, 2, 4, 4]","[2, 3, 3, 2, 4, 4]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, ...","[[3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [3, 3, 3, 3, ...","[[5, 4, 6, 2, 5, 4, 6, 2], [4, 5, 6, 6, 4, 5, ...","[[5, 4, 6, 2], [4, 5, 6, 6], [5, 4, 4, 2]]"
396,"[[1, 1, 0], [1, 1, 0], [0, 1, 0]]","[[1, 1, 0, 0, 1, 1], [1, 1, 0, 0, 1, 1], [0, 1...","[[0, 1, 1], [0, 0, 1], [0, 1, 0]]","[[0, 1, 1, 1, 1, 0], [0, 0, 1, 1, 0, 0], [0, 1...","[0, 0, 0, 0, 0, 1, 0, 0, 1]","[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, ...","[[3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [3, 3, 3, 3, ...","[[0, 0, 1], [0, 0, 0], [0, 1, 0]]","[[0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0], [0, 1..."
397,"[[4, 4, 0], [4, 0, 0], [0, 0, 0], [3, 3, 0], [...","[[6, 6, 0], [6, 0, 0], [0, 0, 6]]","[[4, 4, 0], [0, 0, 0], [4, 0, 4], [3, 3, 3], [...","[[6, 6, 6], [6, 0, 0], [6, 0, 6]]","[0, 0, 4, 0, 3, 0, 0, 4, 4, 3, 0, 3, 4, 4, 0, ...","[0, 6, 6, 6, 6, 6, 6, 6, 6]","[[4, 0, 4], [4, 0, 0], [0, 0, 0], [3, 0, 0], [...","[[6, 0, 6], [6, 0, 0], [0, 6, 0]]","[[0, 4, 0], [4, 0, 4], [4, 0, 0], [3, 3, 3], [...","[[6, 6, 6], [6, 6, 6], [6, 0, 0]]"
398,"[[2, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8], [2, 0, ...","[[2, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]

In [184]:
arc_train["problem_example_1_input"].iloc[0]

[['0', '0', '5'], ['0', '5', '0'], ['5', '0', '0']]

In [9]:
arc_no_id = arc_train.loc[:, arc_train.columns != "problem_id"]
arc_no_id_hf = Dataset.from_pandas(arc_no_id)

In [181]:
arc_no_id_tokenized = arc_no_id.map(
    tokenizer,
    #batched=True,
    #batch_size=10,
)

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [203]:
tokenizer(arc_no_id["problem_example_1_input"][0])

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [213]:
arc_no_id_hf["problem_example_1_input"][0]

[['0', '0', '5'], ['0', '5', '0'], ['5', '0', '0']]

In [197]:
testl

[['0', '0', '5'], ['0', '5', '0'], ['5', '0', '0']]

#### Running the mistral model as is on a few examples to get a baseline score

In [217]:
def chat_with_model(prompt):
    inputs = tokenizer.encode(prompt, return_tensors="pt").to('mps')
    model.to("mps")
    output = model.generate(
        inputs, 
        max_new_tokens=500, 
        do_sample=True, 
        #top_p=0.95, 
        #top_k=60
    )
    
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    
    return response

In [10]:
def grid2str(grid):
    return "\n".join("".join(str(i) for i in j) for j in grid)

num = 4

prompt = (
    f"Here's three examples of a puzzle \n"
    f'Puzzle 1 input: \n {grid2str(arc_no_id_hf["problem_example_1_input"][num])} \n\n'
    f'Puzzle 1 output: \n {grid2str(arc_no_id_hf["problem_example_1_output"][num])} \n\n'
    f'Puzzle 2 input: \n {grid2str(arc_no_id_hf["problem_example_2_input"][num])} \n\n'
    f'Puzzle 2 output: \n {grid2str(arc_no_id_hf["problem_example_2_output"][num])} \n\n'
    f'Puzzle 3 input: \n {grid2str(arc_no_id_hf["problem_example_3_input"][num])} \n\n '
    f'Puzzle 3 output: \n {grid2str(arc_no_id_hf["problem_example_3_output"][num])} \n\n'
    
    f"Based on the examples, could you solve the following? \n\n"
    f'Puzzle to solve: \n {grid2str(arc_no_id_hf["problem_test_input"][num])} \n\n'
    f'Think step-by-step and provide only the final answer.'
)

In [224]:
response = chat_with_model(prompt)
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Here's three examples of a puzzlePuzzle 1: [['0', '0', '5'], ['0', '5', '0'], ['5', '0', '0']]Puzzle 1 solution: [['3', '4', '2'], ['3', '4', '2'], ['3', '4', '2']]Puzzle 2: [['0', '0', '0'], ['0', '0', '0'], ['5', '5', '5']]Puzzle 2 solution: [['3', '3', '3'], ['3', '3', '3'], ['3', '3', '3']]Puzzle 3: [['5', '0', '5'], ['0', '5', '0'], ['0', '0', '0']]Puzzle 3 solution: [['2', '4', '2'], ['2', '4', '2'], ['2', '4', '2']]Based on the examples, could you solve the following:Puzzle to solve: [['0', '5', '0'], ['0', '0', '5'], ['5', '0', '0']]Provide the solution in the same format as the examples above. Think carefully and take a deep breath before answering. And I mean it. We all need to breathe.

Comment

Having problems using this site? Let us know at feedback@mathsbox.com


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3.5-mini-instruct", trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

In [11]:
def chat_with_model(prompt):
    inputs = tokenizer.encode(prompt, return_tensors='pt').to("mps")
    model.to("mps")
    output = model.generate(
        inputs, 
        max_new_tokens=300, 
        #do_sample=True, 
        #top_p=0.95, 
        #top_k=60
    )
    
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    
    return response[len(prompt):].strip()

In [ ]:
response = chat_with_model(prompt)
print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
You are not running the flash-attention implementation, expect numerical differences.


In [56]:
print(grid2str(arc_no_id_hf["problem_test_output"][num]))

2384617
2384617
2384617
2384617
2384617
2384617
2384617


In [54]:
print(grid2str(arc_no_id_hf["problem_test_input"][num]))

2003008004006001007
2003008004006001007
2003008004006001007
2003008004006001007
2003008004006001007
2003008004006001007
2003008004006001007
0000000004006001007
0555555504006001007
0555555504006001007
0555555504006001007
0555555504006001007
0555555504006001007
0555555504006001007
0555555504006001007
0000000004006001007
2003008004006001007
2003008004006001007
2003008004006001007


In [58]:
print(grid2str(arc_no_id_hf["problem_example_1_output"][num]))

213486
213486
213486
213486
213486
213486


In [57]:
arc_no_id_hf["problem_test_input"][num]

[['2',
  '0',
  '0',
  '3',
  '0',
  '0',
  '8',
  '0',
  '0',
  '4',
  '0',
  '0',
  '6',
  '0',
  '0',
  '1',
  '0',
  '0',
  '7'],
 ['2',
  '0',
  '0',
  '3',
  '0',
  '0',
  '8',
  '0',
  '0',
  '4',
  '0',
  '0',
  '6',
  '0',
  '0',
  '1',
  '0',
  '0',
  '7'],
 ['2',
  '0',
  '0',
  '3',
  '0',
  '0',
  '8',
  '0',
  '0',
  '4',
  '0',
  '0',
  '6',
  '0',
  '0',
  '1',
  '0',
  '0',
  '7'],
 ['2',
  '0',
  '0',
  '3',
  '0',
  '0',
  '8',
  '0',
  '0',
  '4',
  '0',
  '0',
  '6',
  '0',
  '0',
  '1',
  '0',
  '0',
  '7'],
 ['2',
  '0',
  '0',
  '3',
  '0',
  '0',
  '8',
  '0',
  '0',
  '4',
  '0',
  '0',
  '6',
  '0',
  '0',
  '1',
  '0',
  '0',
  '7'],
 ['2',
  '0',
  '0',
  '3',
  '0',
  '0',
  '8',
  '0',
  '0',
  '4',
  '0',
  '0',
  '6',
  '0',
  '0',
  '1',
  '0',
  '0',
  '7'],
 ['2',
  '0',
  '0',
  '3',
  '0',
  '0',
  '8',
  '0',
  '0',
  '4',
  '0',
  '0',
  '6',
  '0',
  '0',
  '1',
  '0',
  '0',
  '7'],
 ['0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',

In [61]:
def evaluate_output(predicted_output, expected_output):
    predicted_grid = grid2str(predicted_output)
    return predicted_grid == expected_output

def string_to_grid(grid_str):
    return [[int(char) for char in line] for line in grid_str.strip().split('\n')]

In [70]:
print(prompt)

Here's three examples of a puzzle 
Puzzle 1 input: 

 00000000000000000000000
00000000000000000000000
00000000000000000000000
00000000000000000000000
00000000000000000000000
00000000000000000000000
00000000000000000000000
00000000000000000000000
00000000000000000000000
00000000000000000000000
00000000000000000000000
00800000000000000000000
00000000000000000000000
00000000000000000000000
00000800000000000000000
00000000000000000000000
00000000000000000000000
00000000800000000000000
00000000000000000000000
00000000000000000000000
00000000000000000000000
00000000000000000000000
00000000000000000000000 

Puzzle 1 output: 

 00000000000000000000800
00000000000000000000800
88888888888888888800800
00000000000000000800800
00000000000000000800800
88888888888888800800800
00000000000000800800800
00000000000000800800800
88888888888800800800800
00000000000800800800800
00000000000800800800800
00888888800800800800800
00800000800800800800800
00800000800800800800800
00800800800800800800800
008000008008